In [1]:
# Now, install the dependencies
!pip install datasets
!pip install torch


In [20]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer
from huggingface_hub import notebook_login
from tqdm.auto import tqdm

# --- Step 1: Login to Hugging Face ---
# A login box will appear. Paste your Hugging Face access token here.
# Get a token at: https://huggingface.co/settings/tokens
print("Please log in to your Hugging Face account.")
notebook_login()


Please log in to your Hugging Face account.


In [3]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer
from tqdm import tqdm

# --- Step 2: Configuration ---
PROJECT_PATH = ""
OUTPUT_DIR = os.path.join(PROJECT_PATH, "data")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ✅ Use small token target for testing
TARGET_TOKEN_COUNT = 50_000  # only ~50k tokens (tiny test)
MODEL_NAME = "meta-llama/Llama-3.2-3B"
LANGUAGES = ['en', 'hi']
CONFIG_DATE = '20231101'

print(f"Output directory for text files: {OUTPUT_DIR}")

# --- Step 3: Load the Tokenizer ---
print(f"Loading tokenizer for '{MODEL_NAME}'...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=True)
print("Tokenizer loaded successfully.")

# --- Step 4: Process each language ---
for lang in LANGUAGES:
    print("\n" + "=" * 50)
    print(f"Processing language: {lang}")
    print("=" * 50)

    collected_documents = []
    current_token_count = 0

    dataset_config = f"{CONFIG_DATE}.{lang}"
    streaming_dataset = load_dataset(
        'wikimedia/wikipedia',
        dataset_config,
        split='train',
        streaming=True
    )

    pbar = tqdm(desc=f"Tokens for {lang}", total=TARGET_TOKEN_COUNT)
    for doc in streaming_dataset:
        if current_token_count >= TARGET_TOKEN_COUNT:
            break

        text = doc['text']
        if text:
            # ⚡ Limit text length for quick test
            text = text[:2000]
            num_tokens = len(tokenizer.encode(text))
            collected_documents.append(text)
            current_token_count += num_tokens
            pbar.update(num_tokens)
    pbar.close()

    # --- Step 5: Save to file ---
    output_filepath = os.path.join(OUTPUT_DIR, f"wikipedia_small_{lang}.txt")
    print(f"\nSaving {len(collected_documents)} docs ({current_token_count:,} tokens) to {output_filepath}...")

    with open(output_filepath, 'w', encoding='utf-8') as f:
        f.write("\n\n".join(collected_documents))

    print(f"✅ Successfully saved file for language '{lang}'.")

print("\n🎯 All languages processed successfully (test version)!")


Output directory for text files: data
Loading tokenizer for 'meta-llama/Llama-3.2-3B'...


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

...

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Tokens for en: 50229it [00:01, 39356.90it/s]                           



Saving 119 docs (50,229 tokens) to data/wikipedia_small_en.txt...
✅ Successfully saved file for language 'en'.

Processing language: hi


Tokens for hi: 50440it [00:00, 60170.16it/s]                           


Saving 55 docs (50,440 tokens) to data/wikipedia_small_hi.txt...
✅ Successfully saved file for language 'hi'.

🎯 All languages processed successfully (test version)!


In [4]:
import torch
import os
from transformers import AutoTokenizer
from tqdm.auto import tqdm

# --- Configuration ---
PROJECT_PATH = ""
DATA_DIR = os.path.join(PROJECT_PATH, "data")
MODEL_NAME = "meta-llama/Llama-3.2-3B"
LANGUAGES = ['en', 'hi']

print("🔹 Loading tokenizer...")
# ✅ Use `use_auth_token=True` for gated LLaMA models
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=True)
print("✅ Tokenizer loaded successfully.")

# --- Tokenize each language safely ---
for lang in LANGUAGES:
    # ✅ Match small dataset filenames
    input_filepath = os.path.join(DATA_DIR, f"wikipedia_small_{lang}.txt")
    output_filepath = os.path.join(DATA_DIR, f"id.{lang}.train.small.llama.pt")

    print("\n" + "=" * 60)
    print(f"Processing language: {lang}")
    print(f"Reading from: {input_filepath}")
    print(f"Saving to: {output_filepath}")
    print("=" * 60)

    if not os.path.exists(input_filepath):
        print(f"❌ ERROR: Input file not found at {input_filepath}")
        continue

    token_ids = []

    # --- Stream through the file line by line ---
    with open(input_filepath, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc=f"Tokenizing {lang}"):
            line = line.strip()
            if not line:
                continue

            # Encode the line into token IDs
            ids = tokenizer.encode(line, add_special_tokens=False)
            token_ids.extend(ids)

            # ✅ Flush every 50k tokens (enough for test)
            if len(token_ids) > 50_000:
                torch.save(torch.LongTensor(token_ids), output_filepath)
                token_ids = []

    # --- Save remaining tokens ---
    if token_ids:
        torch.save(torch.LongTensor(token_ids), output_filepath)

    print(f"✅ Saved token tensor for '{lang}' → {output_filepath}")

print("\n🎯 All tokenization steps complete (test mode)!")


🔹 Loading tokenizer...
✅ Tokenizer loaded successfully.

Processing language: en
Reading from: data/wikipedia_small_en.txt
Saving to: data/id.en.train.small.llama.pt


Tokenizing en: 0it [00:00, ?it/s]

✅ Saved token tensor for 'en' → data/id.en.train.small.llama.pt

Processing language: hi
Reading from: data/wikipedia_small_hi.txt
Saving to: data/id.hi.train.small.llama.pt


Tokenizing hi: 0it [00:00, ?it/s]

✅ Saved token tensor for 'hi' → data/id.hi.train.small.llama.pt

🎯 All tokenization steps complete (test mode)!


**activation.py**

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm

MODEL_NAME = "meta-llama/Llama-3.2-3B"
LANGUAGES = ['en', 'hi']
BATCH_SIZE = 2
MAX_LENGTH = 1024

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

num_layers = model.config.num_hidden_layers
intermediate_size = model.config.intermediate_size  # 8192

# Prepare counter
over_zero = torch.zeros(num_layers, intermediate_size, dtype=torch.int32).to('cuda')

# Hook function for full MLP output
def mlp_hook(layer_idx):
    def hook(module, input, output):
        # output = (F.silu(gate_proj(x)) * up_proj(x))
        activation = (torch.nn.functional.silu(module.gate_proj(input[0])) * module.up_proj(input[0])).float()
        over_zero[layer_idx, :activation.size(-1)] += (activation > 0).sum(dim=(0,1))
    return hook

for i, layer in enumerate(model.model.layers):
    layer.mlp.register_forward_hook(mlp_hook(i))


# Example: processing dummy input
for lang in LANGUAGES:
    # Load token IDs (already prepared)
    tensor_path = f"data/id.{lang}.train.small.llama.pt"
    ids = torch.load(tensor_path).to('cuda')
    total_tokens = ids.size(0)

    for start in tqdm(range(0, total_tokens, BATCH_SIZE * MAX_LENGTH), desc=f"Processing {lang}"):
        batch_ids = []
        for b in range(BATCH_SIZE):
            s = start + b * MAX_LENGTH
            if s >= total_tokens:
                break
            e = min(s + MAX_LENGTH, total_tokens)
            chunk = ids[s:e]
            if chunk.size(0) < MAX_LENGTH:
                chunk = torch.cat([chunk, torch.zeros(MAX_LENGTH - chunk.size(0), dtype=chunk.dtype, device='cuda')])
            batch_ids.append(chunk)
        if not batch_ids:
            continue

        batch_ids = torch.stack(batch_ids, dim=0)
        attention_mask = (batch_ids != 0).long()
        with torch.no_grad():
            _ = model(input_ids=batch_ids, attention_mask=attention_mask)

    # Save activations
    output_path = f"data/activation.{lang}.train.small.llama3b.pt"
    torch.save({'over_zero': over_zero.cpu()}, output_path)
    print(f"✅ Saved activations for {lang} → {output_path}")


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing en:   0%|          | 0/25 [00:00<?, ?it/s]

✅ Saved activations for en → data/activation.en.train.small.llama3b.pt


Processing hi:   0%|          | 0/25 [00:00<?, ?it/s]

✅ Saved activations for hi → data/activation.hi.train.small.llama3b.pt


In [14]:
import torch
import pandas as pd
import numpy as np

# --- Load the activation file ---
file_path = 'data/activation.en.train.small.llama3b.pt'
data = torch.load(file_path)

over_zero = data['over_zero'].cpu()  # 28 x 8192

# --- Convert to NumPy for easier printing ---
matrix = over_zero.numpy()

# Option 1: Print the full matrix (will flood the console)
np.set_printoptions(threshold=np.inf)  # show all elements
print(matrix)

# Option 2: Save to CSV (recommended for inspection)
df = pd.DataFrame(matrix)
csv_path = file_path.replace(".pt", ".csv")
df.to_csv(csv_path, index=False)
print(f"✅ over_zero tensor saved to CSV: {csv_path}")


[[23932 26541 22889 24701 23519 27044 25439 23548 25524 24850 24826 23254
  24269 26144 25021 28499 23397 23898 21103 15299 22757 23809 28329 29479
  25333 26107 26467 24290 16758 20717 27334 26987 25142 24184 24846 22554
  25379 24863 23666 26670 26215 26961 18433 25982 26262 23018 23390 28025
  21107 25944 24672 23868 24326 26893 20601 24045 25860 23518 24697 25313
  24295 24017 28231 21469 25444 22262 27568 29042 25425 23850 24058 23742
  24650 32059 27497 19029 27921 23705 24891 23994 25178 26478 22861 27565
  25438 25237 24544 21073 22700 28977 23930 29081 26658 26056 24514 26052
  25867 27961 28871 25900 26037 23834 25332 25220 29523 24209 27349 27180
  23035 26784 23190 24108 22776 24458 26864 32314 21390 25111 22866 24693
  28793 26708 28628 26085 36137 29657 24441 26753 24714 24834 26537 25747
  25070 22423 19789 24467 27906 24075 25629 28069 21106 25500 24264 27384
  26480 24181 22692 24064 21114 25436 23282 20788 23308 23671 27825 21057
  28586 22676 22819 24566 29886 30257 

In [15]:
file_path = 'data/activation.hi.train.small.llama3b.pt'
data = torch.load(file_path)

over_zero = data['over_zero'].cpu()  # 28 x 8192

# --- Convert to NumPy for easier printing ---
matrix = over_zero.numpy()

# Option 1: Print the full matrix (will flood the console)
np.set_printoptions(threshold=np.inf)  # show all elements
print(matrix)

# Option 2: Save to CSV (recommended for inspection)
df = pd.DataFrame(matrix)
csv_path = file_path.replace(".pt", ".csv")
df.to_csv(csv_path, index=False)
print(f"✅ over_zero tensor saved to CSV: {csv_path}")

[[ 48977  48929  48893  47232  49186  54014  56347  50663  49371  50221
   50945  44292  50944  50756  49148  51463  44153  48207  49175  32250
   51229  49452  58317  52660  51801  52279  51629  51028  35017  42408
   53034  51591  50937  48683  51656  48950  53826  55542  48377  53131
   48913  50072  41433  51833  47041  53692  53152  56275  56793  51682
   45953  49337  50477  49291  45650  49876  47547  52352  47262  52187
   53131  45700  47135  41726  50519  45750  55921  57756  50432  49658
   45650  49428  50909  55833  53370  38629  51166  47287  50761  51047
   49182  57353  45590  54493  51522  52489  46856  42908  47587  52294
   47875  62106  53982  49361  50330  51898  52341  51621  51820  54518
   50183  47900  52536  47698  54263  50498  51152  53102  48912  57754
   48239  47060  49570  47105  51210  58571  48258  51404  48395  54134
   52787  50233  54190  51936  65179  50833  49460  51527  51290  47722
   53954  53611  49694  50901  45003  51149  56067  54809  49033

**identify.py**

In [26]:
import torch
import os

# --- Configuration ---
languages = ['en', 'hi']  # your hi-en dataset
activation_dir = 'data'
output_dir = 'activation_mask'
os.makedirs(output_dir, exist_ok=True)

top_rate = 0.01           # fraction of neurons to select (entropy-based)
filter_rate = 0.95        # top 5% neurons threshold
activation_bar_ratio = 0.95  # top 5% activation threshold

# --- Load activation data ---
over_zero = []
for lang in languages:
    file_path = os.path.join(activation_dir, f'activation.{lang}.train.small.llama3b.pt')
    data = torch.load(file_path)
    over_zero.append(data['over_zero'])

# shape: [layers, neurons, languages]
over_zero = torch.stack(over_zero, dim=-1)
num_layers, intermediate_size, lang_num = over_zero.size()
print(f"{lang_num} languages, {num_layers} layers, {intermediate_size} neurons per layer")

# --- Compute activation probabilities across languages ---
activation_probs = over_zero / over_zero.sum(dim=-1, keepdim=True)
activation_probs[torch.isnan(activation_probs)] = 0

# --- Compute entropy across languages ---
log_probs = torch.where(activation_probs > 0, activation_probs.log(), 0)
entropy = -torch.sum(activation_probs * log_probs, dim=-1)  # shape: [layers, neurons]

# --- Filter neurons with very low activation ---
flattened_probs = activation_probs.flatten()
top_prob_value = flattened_probs.kthvalue(round(len(flattened_probs) * filter_rate)).values.item()
active_neurons = (activation_probs > top_prob_value).sum(dim=-1)
entropy[active_neurons == 0] = torch.inf  # ignore inactive neurons

# --- Select top neurons by smallest entropy (language-specific) ---
flattened_entropy = entropy.flatten()
top_entropy_count = round(len(flattened_entropy) * top_rate)
entropy_values, index = flattened_entropy.topk(top_entropy_count, largest=False)  # smallest entropy = language-specific

row_index = index // entropy.size(1)
col_index = index % entropy.size(1)

# --- Determine which language dominates each top neuron ---
# For each selected neuron (layer,row_index,col_index),
# check which language has the higher activation probability
dominant_lang = []
for r, c in zip(row_index.tolist(), col_index.tolist()):
    probs = activation_probs[r, c]  # shape [languages]
    winner = probs.argmax().item()  # 0 -> en, 1 -> hi
    dominant_lang.append(winner)

dominant_lang = torch.tensor(dominant_lang)
counts = torch.bincount(dominant_lang, minlength=len(languages))

total = counts.sum().item()
print("\n🔍 Top 1% language dominance based on entropy:")
for i, lang in enumerate(languages):
    pct = 100 * counts[i].item() / total if total > 0 else 0
    print(f"   {lang.upper():>3}: {counts[i].item()} neurons ({pct:.2f}%)")

# --- Build and save per-language masks separately (same as before) ---
for lang_id, lang_name in enumerate(languages):
    lang_probs = activation_probs[:, :, lang_id]
    selected_mask = torch.zeros_like(lang_probs, dtype=torch.bool)
    selected_mask[row_index, col_index] = True
    activation_bar = flattened_probs.kthvalue(round(len(flattened_probs) * activation_bar_ratio)).values.item()
    selected_mask &= (lang_probs > activation_bar)

    layer_indices = []
    for layer in range(num_layers):
        neurons = torch.where(selected_mask[layer])[0]
        layer_indices.append(neurons)

    output_file = os.path.join(output_dir, f"llama3b_{lang_name}_mask.pt")
    torch.save(layer_indices, output_file)
    print(f"✅ Activation mask for {lang_name} saved to: {output_file}")


2 languages, 28 layers, 8192 neurons per layer

🔍 Top 1% language dominance based on entropy:
    EN: 0 neurons (0.00%)
    HI: 2294 neurons (100.00%)
✅ Activation mask for en saved to: activation_mask/llama3b_en_mask.pt
✅ Activation mask for hi saved to: activation_mask/llama3b_hi_mask.pt


In [19]:
import torch
import os

# --- Configuration ---
output_dir = 'activation_mask'
languages = ['en', 'hi']

for lang in languages:
    file_path = os.path.join(output_dir, f'llama3b_{lang}_mask.pt')

    # Load the saved mask
    layer_masks = torch.load(file_path)

    print(f"\n--- Activation mask for {lang.upper()} ---")
    print("layer-wise neuron indices = [")

    for i, neuron_tensor in enumerate(layer_masks):
        print(f"    layer {i}: tensor({neuron_tensor.tolist()})")

    print("]\n")



--- Activation mask for EN ---
layer-wise neuron indices = [
    layer 0: tensor([])
    layer 1: tensor([])
    layer 2: tensor([])
    layer 3: tensor([])
    layer 4: tensor([])
    layer 5: tensor([])
    layer 6: tensor([])
    layer 7: tensor([])
    layer 8: tensor([])
    layer 9: tensor([])
    layer 10: tensor([])
    layer 11: tensor([])
    layer 12: tensor([])
    layer 13: tensor([])
    layer 14: tensor([])
    layer 15: tensor([])
    layer 16: tensor([])
    layer 17: tensor([])
    layer 18: tensor([])
    layer 19: tensor([])
    layer 20: tensor([])
    layer 21: tensor([])
    layer 22: tensor([])
    layer 23: tensor([])
    layer 24: tensor([])
    layer 25: tensor([])
    layer 26: tensor([])
    layer 27: tensor([])
]


--- Activation mask for HI ---
layer-wise neuron indices = [
    layer 0: tensor([48, 268, 513, 597, 988, 1121, 1365, 1652, 1934, 2353, 2788, 2990, 3936, 4278, 4723, 5281, 5758, 5943, 6197, 6237, 6248, 6384, 7374, 7418, 7503, 8004])
    layer 1